# ScoreCard Complete Pipeline

End-to-end pipeline including:
1. **Data Download** - SQL Server
2. **NLP Enrichment** - spaCy text processing
3. **Model Training** - H1/H2 horizon predictions
4. **Elasticsearch Indexing** - RAG embeddings
5. **GPT Enrichment** - Generate justifications for predictions
6. **SQL Upload** - Push results back to database

---
## 1. Imports and Configuration

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from scorecard import (
    # Core
    ScoreCardConfig,
    ScoreCardState,
    ConnectionManager,
    ScoreCardPipeline,
    ScoreCardRag,
    run_pipeline,
    # Upload
    upload_predictions_to_sql,
    build_upload_table,
    # State persistence
    save_state,
)

print("Imports successful!")

In [ ]:
# Configuration
config = ScoreCardConfig(
    # Data source
    sql_download=True,
    
    # Pipeline stages
    enable_nlp=True,
    build_models=True,
    run_predictions=True,
    build_rag=True,
    
    # Model keys (use predetermined for speed)
    default_model_key_h1="complete_main_words_only | no_downsample_weighted | count | {0: 0.5, 1: 1.35, 2: 1.15}",
    default_model_key_h2="complete_main_words_only | no_downsample_weighted | count | {0: 0.5, 1: 1.35, 2: 1.15}",
    
    training_length=5,
)

print("Configuration ready!")

---
## 2. Run Core Pipeline (NLP + Modeling + ES Indexing)

In [ ]:
# Run the full pipeline
# This handles: SQL download, NLP, model training, predictions, RAG embeddings
state, pipeline, rag = run_pipeline(
    sql_download=config.sql_download,
    enable_nlp=config.enable_nlp,
    build_models=config.build_models,
    run_predictions=config.run_predictions,
    build_rag=config.build_rag,
)

# Get connection manager reference
conn = pipeline.conn

print("\n" + "="*60)
print("CORE PIPELINE COMPLETE")
print("="*60)

In [ ]:
# Quick summary
print(f"Enriched notes: {len(state.enriched_df):,}")
print(f"Predictions: {len(state.predictions_df):,}")
print(f"Complete dataset: {len(state.complete_df):,}")
for h, key in state.best_model_key_by_horizon.items():
    print(f"H{h} model: {key[:50]}...")

---
## 3. GPT Enrichment - Generate Justifications

This step calls GPT to explain WHY the model made each prediction.

In [ ]:
# GPT Enrichment settings
GPT_LIMIT = None          # Set to None for ALL notes, or a number to limit (e.g., 100)
GPT_MAX_WORKERS = 4       # Parallel threads for GPT calls
GPT_BACKOFF = 1.5         # Backoff multiplier for retries

print("="*60)
print("GPT ENRICHMENT CONFIGURATION")
print("="*60)
print(f"Limit: {GPT_LIMIT if GPT_LIMIT else 'ALL NOTES'}")
print(f"Max workers: {GPT_MAX_WORKERS}")
print(f"Total notes to process: {len(state.complete_df) if GPT_LIMIT is None else min(GPT_LIMIT, len(state.complete_df))}")

In [ ]:
# Ensure RAG object is ready
if rag is None:
    print("Creating RAG object...")
    rag = ScoreCardRag(config=config, state=state, conn=conn)
    
print(f"RAG index: {config.rag_index}")
print(f"GPT model: {rag.gpt_model}")

In [ ]:
# Run GPT justification pass
print("="*60)
print("STARTING GPT ENRICHMENT")
print("="*60)
print("This will generate GPT explanations for each prediction...")
print("(This may take a while depending on the number of notes)\n")

rag.run_gpt_justification_pass(
    limit=GPT_LIMIT,
    max_attempts=6,
    backoff=GPT_BACKOFF,
)

In [ ]:
# Verify GPT enrichment by sampling a few
print("="*60)
print("GPT ENRICHMENT VERIFICATION")
print("="*60)

es = conn.es_client

# Check how many have justifications
query = {
    "query": {
        "bool": {
            "must": {"exists": {"field": "justification"}},
            "must_not": {"term": {"justification.keyword": ""}}
        }
    }
}
result = es.count(index=config.rag_index, body=query)
print(f"\nNotes with GPT justifications: {result['count']:,}")

# Show a sample
sample_query = {
    "size": 3,
    "query": query["query"],
    "_source": ["sid_key", "SID", "justification"]
}
samples = es.search(index=config.rag_index, body=sample_query)

print("\nSample justifications:")
for hit in samples['hits']['hits']:
    src = hit['_source']
    print(f"\n--- {src['sid_key']} (SID {src['SID']}) ---")
    print(src.get('justification', 'N/A')[:500] + "...")

---
## 4. Prepare Upload DataFrame

Merge predictions with GPT justifications for SQL upload.

In [ ]:
import pandas as pd

def fetch_all_justifications(es, index, batch_size=1000):
    """
    Fetch all justifications from ES using scroll API.
    """
    justifications = {}
    
    query = {
        "size": batch_size,
        "query": {"match_all": {}},
        "_source": ["sid_key", "justification"]
    }
    
    # Initial search
    result = es.search(index=index, body=query, scroll='2m')
    scroll_id = result['_scroll_id']
    hits = result['hits']['hits']
    
    while hits:
        for hit in hits:
            src = hit['_source']
            sid_key = src.get('sid_key')
            justification = src.get('justification', '')
            if sid_key and justification:
                justifications[sid_key] = justification
        
        # Get next batch
        result = es.scroll(scroll_id=scroll_id, scroll='2m')
        hits = result['hits']['hits']
    
    # Clear scroll
    es.clear_scroll(scroll_id=scroll_id)
    
    return justifications

print("Fetching all GPT justifications from ES...")
justifications = fetch_all_justifications(conn.es_client, config.rag_index)
print(f"Retrieved {len(justifications):,} justifications")

In [ ]:
# Merge justifications into predictions DataFrame
upload_df = state.complete_df.copy()

# Add GPT justification column
upload_df['GPT_Justification'] = upload_df['sid_key'].map(justifications).fillna('')

print(f"Upload DataFrame: {upload_df.shape}")
print(f"Notes with justifications: {(upload_df['GPT_Justification'] != '').sum():,}")

# Show sample
cols_to_show = ['sid_key', 'SID', 'predicted_color', 'prob_green', 'prob_yellow', 'prob_red', 'GPT_Justification']
available_cols = [c for c in cols_to_show if c in upload_df.columns]
upload_df[available_cols].head(3)

---
## 5. SQL Upload

In [ ]:
# Build the upload table with proper schema
print("="*60)
print("PREPARING SQL UPLOAD")
print("="*60)

upload_table = build_upload_table(upload_df)
print(f"\nUpload table shape: {upload_table.shape}")
print(f"Columns: {list(upload_table.columns)}")

In [ ]:
# Preview before upload
print("\nUpload Preview (first 5 rows):")
upload_table.head()

In [ ]:
# DRY RUN - Preview what would be uploaded
print("="*60)
print("SQL UPLOAD - DRY RUN")
print("="*60)

upload_predictions_to_sql(
    df=upload_table,
    config=config,
    table_name="Model_Predictions",
    dry_run=True,  # <-- Set to False for actual upload
)

In [ ]:
# ACTUAL UPLOAD - Uncomment to execute
# print("="*60)
# print("SQL UPLOAD - EXECUTING")
# print("="*60)

# upload_predictions_to_sql(
#     df=upload_table,
#     config=config,
#     table_name="Model_Predictions",
#     dry_run=False,  # <-- Actual upload!
# )

# print("\nUpload complete!")

---
## 6. Save State (Optional)

Save the pipeline state for later analysis without re-running.

In [ ]:
# Save state for later use
save_state(
    state=state,
    pipeline=pipeline,
    rag=rag,
    config=config,
    conn=conn,
    path="./pipeline_state.pkl",
    include_models=True,
    verbose=True,
)

---
## 7. Pipeline Summary

In [ ]:
print("="*60)
print("COMPLETE PIPELINE SUMMARY")
print("="*60)

print(f"\n1. DATA")
print(f"   - Raw notes downloaded: {len(state.details_df):,}")
print(f"   - Enriched notes: {len(state.enriched_df):,}")

print(f"\n2. MODELS")
for h, key in state.best_model_key_by_horizon.items():
    print(f"   - H{h}: {key[:50]}...")

print(f"\n3. PREDICTIONS")
print(f"   - Total predictions: {len(state.complete_df):,}")
for h, df in state.predictions_df_by_horizon.items():
    if df is not None:
        print(f"   - H{h} predictions: {len(df):,}")

print(f"\n4. GPT ENRICHMENT")
print(f"   - Notes with justifications: {len(justifications):,}")

print(f"\n5. UPLOAD")
print(f"   - Upload table rows: {len(upload_table):,}")
print(f"   - Status: Ready (run actual upload cell to execute)")

print(f"\n" + "="*60)
print("PIPELINE COMPLETE!")
print("="*60)